Link to kaggle competition https://www.kaggle.com/c/santander-customer-transaction-prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from keras import models
from keras import layers
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from keras import optimizers
from keras.constraints import maxnorm
from sklearn.preprocessing import normalize

Using TensorFlow backend.


In [2]:
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')

In [3]:
#Checking null values in data
training_data.isnull().sum().sum()

0

In [4]:
testing_data.isnull().sum().sum()

0

In [5]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [6]:
#Checking which is the object variable
training_data.describe(include = ['O'])

,ID_code
count,200000
unique,200000
top,train_150932
freq,1


In [7]:
training_data.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [8]:
training_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [9]:
#This looks like a inbalance dataset

training_data['target'].value_counts(normalize = True)

0    0.89951
1    0.10049
Name: target, dtype: float64

In [10]:
training_data.shape

(200000, 202)

In [11]:
testing_data.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [12]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 201 entries, ID_code to var_199
dtypes: float64(200), object(1)
memory usage: 306.7+ MB


In [13]:
testing_data.shape

(200000, 201)

In [14]:
training_data.tail()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001
199999,train_199999,0,10.8762,-5.7105,12.1183,8.0328,11.5577,0.3488,5.2839,15.2058,...,8.9842,1.6893,0.1276,0.3766,15.2101,-2.4907,-2.2342,8.1857,12.1284,0.1385


In [15]:
training_predictors = training_data.drop(['target', 'ID_code'], axis = 1)

training_target = training_data['target']

In [16]:
train_X, val_X, train_y, val_y = train_test_split(training_predictors, training_target, test_size = 0.25,
                                                 stratify = training_target, random_state = 0) 

In [17]:
model = models.Sequential()
model.add(layers.Dense(150, activation='relu', input_shape=(training_predictors.shape[1],)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#history = 
model.fit(train_X, 
            train_y,
            epochs=20,
            batch_size=128,
            validation_data=(val_X, val_y))

Train on 150000 samples, validate on 50000 samples
Epoch 1/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.3037 - acc: 0.8995 - val_loss: 0.2681 - val_acc: 0.9039
Epoch 2/20
150000/150000 [==============================] - 4s 27us/step - loss: 0.2674 - acc: 0.9045 - val_loss: 0.2639 - val_acc: 0.9067
Epoch 3/20
150000/150000 [==============================] - 4s 26us/step - loss: 0.2632 - acc: 0.9056 - val_loss: 0.2589 - val_acc: 0.9037
Epoch 4/20
150000/150000 [==============================] - 3s 23us/step - loss: 0.2602 - acc: 0.9062 - val_loss: 0.2706 - val_acc: 0.9035
Epoch 5/20
150000/150000 [==============================] - 3s 23us/step - loss: 0.2586 - acc: 0.9076 - val_loss: 0.3094 - val_acc: 0.9050
Epoch 6/20
150000/150000 [==============================] - 3s 22us/step - loss: 0.2580 - acc: 0.9076 - val_loss: 0.2548 - val_acc: 0.9080
Epoch 7/20
150000/150000 [==============================] - 3s 22us/step - loss: 0.2568 - acc: 0.9082 - val_loss: 0

In [18]:
std = StandardScaler()
train_X = std.fit_transform(train_X)
val_X = std.fit_transform(val_X)
testing_predictor_transformed = std.transform(testing_data.drop(['ID_code'], axis = 1))

In [19]:
model = models.Sequential()
model.add(layers.Dense(150, activation='relu', input_shape=(training_predictors.shape[1],)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_X, 
            train_y,
            epochs=20,
            batch_size=128,
            validation_data=(val_X, val_y))

Train on 150000 samples, validate on 50000 samples
Epoch 1/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.2515 - acc: 0.9085 - val_loss: 0.2436 - val_acc: 0.9106
Epoch 2/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.2327 - acc: 0.9143 - val_loss: 0.2452 - val_acc: 0.9104
Epoch 3/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.2224 - acc: 0.9179 - val_loss: 0.2460 - val_acc: 0.9101
Epoch 4/20
150000/150000 [==============================] - 3s 23us/step - loss: 0.2114 - acc: 0.9217 - val_loss: 0.2589 - val_acc: 0.9091
Epoch 5/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.1987 - acc: 0.9272 - val_loss: 0.2714 - val_acc: 0.9071
Epoch 6/20
150000/150000 [==============================] - 4s 24us/step - loss: 0.1853 - acc: 0.9321 - val_loss: 0.2907 - val_acc: 0.9043
Epoch 7/20
150000/150000 [==============================] - 3s 23us/step - loss: 0.1730 - acc: 0.9372 - val_loss: 0

In [20]:
sgd = optimizers.SGD(lr = 0.05)
model = models.Sequential()

model.add(layers.Dense(250, activation='relu', input_shape=(training_predictors.shape[1],)))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(180, activation='relu'))
model.add(layers.Dropout(0.40))
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(70, activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(30, activation = 'relu'))
model.add(layers.Dense(10, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_X, 
            train_y,
            epochs=20,
            batch_size=128,
            validation_data=(val_X, val_y))

Train on 150000 samples, validate on 50000 samples
Epoch 1/20
150000/150000 [==============================] - 9s 59us/step - loss: 0.3330 - acc: 0.8983 - val_loss: 0.3188 - val_acc: 0.8995
Epoch 2/20
150000/150000 [==============================] - 8s 56us/step - loss: 0.3007 - acc: 0.8995 - val_loss: 0.2747 - val_acc: 0.8995
Epoch 3/20
150000/150000 [==============================] - 8s 55us/step - loss: 0.2757 - acc: 0.8999 - val_loss: 0.2491 - val_acc: 0.9042
Epoch 4/20
150000/150000 [==============================] - 8s 54us/step - loss: 0.2626 - acc: 0.9023 - val_loss: 0.2509 - val_acc: 0.9107
Epoch 5/20
150000/150000 [==============================] - 8s 55us/step - loss: 0.2560 - acc: 0.9036 - val_loss: 0.2521 - val_acc: 0.9110
Epoch 6/20
150000/150000 [==============================] - 8s 56us/step - loss: 0.2524 - acc: 0.9054 - val_loss: 0.2457 - val_acc: 0.9089
Epoch 7/20
150000/150000 [==============================] - 8s 55us/step - loss: 0.2489 - acc: 0.9061 - val_loss: 0

In [21]:
sgd = optimizers.SGD(lr = 0.07)
np.random.seed(7)
model = models.Sequential()

model.add(layers.Dense(250, activation='relu',kernel_initializer = 'normal',kernel_constraint = maxnorm(3), input_shape=(training_predictors.shape[1],)))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(180,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation='relu'))
model.add(layers.Dropout(0.40))
model.add(layers.Dense(120,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(70,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(30,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(10,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_X, 
            train_y,
            epochs=20,
            batch_size=128,
            validation_data=(val_X, val_y))

Train on 150000 samples, validate on 50000 samples
Epoch 1/20
150000/150000 [==============================] - 10s 64us/step - loss: 0.3366 - acc: 0.8990 - val_loss: 0.3263 - val_acc: 0.8995
Epoch 2/20
150000/150000 [==============================] - 9s 61us/step - loss: 0.3277 - acc: 0.8995 - val_loss: 0.3259 - val_acc: 0.8995
Epoch 3/20
150000/150000 [==============================] - 9s 62us/step - loss: 0.3264 - acc: 0.8995 - val_loss: 0.3248 - val_acc: 0.8995
Epoch 4/20
150000/150000 [==============================] - 10s 63us/step - loss: 0.3188 - acc: 0.8996 - val_loss: 0.2803 - val_acc: 0.8995
Epoch 5/20
150000/150000 [==============================] - 9s 62us/step - loss: 0.2718 - acc: 0.9031 - val_loss: 0.2458 - val_acc: 0.9051
Epoch 6/20
150000/150000 [==============================] - 9s 61us/step - loss: 0.2549 - acc: 0.9060 - val_loss: 0.2409 - val_acc: 0.9092
Epoch 7/20
150000/150000 [==============================] - 9s 61us/step - loss: 0.2495 - acc: 0.9067 - val_loss:

In [22]:
np.random.seed(7)
model = models.Sequential()

model.add(layers.Dense(250, activation='relu',kernel_initializer = 'normal',kernel_constraint = maxnorm(3), input_shape=(training_predictors.shape[1],)))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(180,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation='relu'))
model.add(layers.Dropout(0.40))
model.add(layers.Dense(120,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(70,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(30,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(10,kernel_initializer = 'normal',kernel_constraint = maxnorm(3), activation = 'relu'))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_X, 
            train_y.values,
            epochs=9,
            batch_size=128,
            validation_data=(val_X, val_y.values))

Train on 150000 samples, validate on 50000 samples
Epoch 1/9
150000/150000 [==============================] - 10s 65us/step - loss: 0.3366 - acc: 0.8990 - val_loss: 0.3263 - val_acc: 0.8995
Epoch 2/9
150000/150000 [==============================] - 9s 61us/step - loss: 0.3277 - acc: 0.8995 - val_loss: 0.3259 - val_acc: 0.8995
Epoch 3/9
150000/150000 [==============================] - 9s 61us/step - loss: 0.3264 - acc: 0.8995 - val_loss: 0.3248 - val_acc: 0.8995
Epoch 4/9
150000/150000 [==============================] - 11s 71us/step - loss: 0.3188 - acc: 0.8996 - val_loss: 0.2806 - val_acc: 0.8995
Epoch 5/9
150000/150000 [==============================] - 10s 65us/step - loss: 0.2719 - acc: 0.9030 - val_loss: 0.2457 - val_acc: 0.9051
Epoch 6/9
150000/150000 [==============================] - 10s 67us/step - loss: 0.2550 - acc: 0.9059 - val_loss: 0.2408 - val_acc: 0.9091
Epoch 7/9
150000/150000 [==============================] - 9s 60us/step - loss: 0.2495 - acc: 0.9067 - val_loss: 0.24

In [23]:
predictions = model.predict(testing_predictor_transformed)

In [24]:
# got score of 77.9
answers = pd.DataFrame({'ID_code':testing_data['ID_code'], 'target':predictions.reshape(200000,)})
answers.to_csv('deeppredictions.csv',index = False)